# TensorFlow

## グラフの作成とセッション内での実行

In [2]:
import tensorflow as tf

In [3]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

このコードは計算グラフを作るだけで、実際変数でさえ初期化されていない。このグラフを評価するには、`TensorFlow セッション`を開き、それを使用して変数を初期化し、fを評価する。

In [5]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
result

42

In [6]:
sess.close()

In [9]:
# 別のやり方1
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
result

42

In [11]:
# 別のやり方2(Jupyterはこれを推奨)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
result

42

`TensolFlow`は一般的に二つの部分に分割される。　一つ目が計算グラフを作る`構築フェーズ`、二つ目がそれを実行する`実行フェーズ`。

## グラフの管理
作成したノートは、自動的にデフォルトのグラフに追加される。

In [13]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

複数の独立したグラフを管理したい場合は、withブロックでそれを一時的にデフォルトグラフにする

In [15]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [16]:
x2.graph is graph

True

In [17]:
x2.graph is tf.get_default_graph()

False

## ノード値のライフサイクル

In [19]:
w = tf.constant(3)

x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


## TensorFlowによる線形回帰

In [20]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [21]:
housing = fetch_california_housing()
m, n = housing.data.shape

I0822 14:09:19.610981 4634498496 california_housing.py:115] Downloading Cal. housing from https://ndownloader.figshare.com/files/5976036 to /Users/shimizukousuke/scikit_learn_data


In [22]:
m, n

(20640, 8)

In [28]:
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data] # バイアス項1を付与
housing_data_plus_bias[:2]

array([[ 1.00000000e+00,  8.32520000e+00,  4.10000000e+01,
         6.98412698e+00,  1.02380952e+00,  3.22000000e+02,
         2.55555556e+00,  3.78800000e+01, -1.22230000e+02],
       [ 1.00000000e+00,  8.30140000e+00,  2.10000000e+01,
         6.23813708e+00,  9.71880492e-01,  2.40100000e+03,
         2.10984183e+00,  3.78600000e+01, -1.22220000e+02]])

In [37]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

In [38]:
XT = tf.transpose(X)

In [39]:
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [40]:
with tf.Session() as sess:
    theta_value = theta.eval()
theta_value

array([[-3.6894890e+01],
       [ 4.3661433e-01],
       [ 9.4453208e-03],
       [-1.0704148e-01],
       [ 6.4345831e-01],
       [-3.9632569e-06],
       [-3.7880042e-03],
       [-4.2093179e-01],
       [-4.3400639e-01]], dtype=float32)

NumPyで正規方程式を計算するのに比べTensorFlowはGPUが使える場合はGPUでこれを実行してくれる